## グリッドサーチ

決定木の深さのようなパラメータが複数ある場合、最も良い性能を得ることのできる組み合わせを探索する必要があるが、その作業は煩雑になる。パラメータを列挙するだけで、機械的にパラメータを組み合わせて検証を実行する仕組みとして、グリッドサーチがある。

In [1]:
import pandas as pd
import numpy as np
loan = pd.read_csv('data/loan_data.csv')

# 使用しない列を削除
df = loan.drop(['Unnamed: 0', 'status', 'loan_amnt', 'term', 'annual_inc', 'dti',
                  'revol_bal', 'revol_util', 'purpose', 'home_ownership',
                  'delinq_2yrs_zero', 'pub_rec_zero', 'open_acc', 'grade'], axis=1)

# ダミー変数化
df['outcome'] = df['outcome'].map(lambda x: 1 if x == 'default' else 0)
df = pd.get_dummies(data=df)

# 説明変数、目的変数を作成
y = 'outcome'
X = df.drop([y], axis=1)
Y = df[y]

### グリッドの設定とグリッドサーチ

In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

# 決定木分類器を作成
model = DecisionTreeClassifier(criterion='entropy')

# 探索パラメータ
# - 決定木の深さ
param_depth = list(range(4, 11, 1))

# グリッドの設定
# - 引数：max_depth, パラメータのグリッド：param_depth
grid = [{'max_depth': param_depth}]

# グリッドサーチの作成、実行 n_jobsは並列実行数
gs = GridSearchCV(model, param_grid=grid, scoring='roc_auc', cv=5, n_jobs=-1)
gs.fit(X, Y)

# 結果の表示
print('best AUC={:.3f}'.format(gs.best_score_))
print('parameter:', gs.best_params_)

best AUC=0.672
parameter: {'max_depth': 6}


cv_results_ からグリッドサーチの実行結果を取得可能。

In [3]:
gs.cv_results_

{'mean_fit_time': array([0.05683794, 0.0752008 , 0.0651319 , 0.07310076, 0.0829546 ,
        0.09291801, 0.08813701]),
 'std_fit_time': array([0.00639828, 0.00893945, 0.00485882, 0.00747136, 0.00746314,
        0.00653196, 0.01002689]),
 'mean_score_time': array([0.00364285, 0.00410423, 0.00558925, 0.00600204, 0.00366998,
        0.00600772, 0.00329919]),
 'std_score_time': array([5.52348401e-04, 6.16763466e-04, 1.82781362e-03, 5.35576676e-03,
        4.91495538e-04, 2.43443573e-03, 9.57416583e-05]),
 'param_max_depth': masked_array(data=[4, 5, 6, 7, 8, 9, 10],
              mask=[False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_depth': 4},
  {'max_depth': 5},
  {'max_depth': 6},
  {'max_depth': 7},
  {'max_depth': 8},
  {'max_depth': 9},
  {'max_depth': 10}],
 'split0_test_score': array([0.59929502, 0.60052532, 0.60602446, 0.60490706, 0.60619441,
        0.60383081, 0.59875091]),
 'split1_test_score': array([0.6577

見やすくするために、pandas DataFrame に変換し、AUCが上位のパラメータ（深さ）を表示。

In [4]:
df = pd.DataFrame(gs.cv_results_).sort_values('mean_test_score', ascending=False)
df[['mean_test_score', 'params']].head(5)

,mean_test_score,params
2,0.672057,{'max_depth': 6}
3,0.671793,{'max_depth': 7}
1,0.670079,{'max_depth': 5}
4,0.669167,{'max_depth': 8}
0,0.666962,{'max_depth': 4}


### 複数のパラメータの指定

DecisionTreeClassifier で分割基準も変更。

In [5]:
# 決定木分類器を作成
model = DecisionTreeClassifier()

# 探索パラメータ
# - 決定木の深さ
param_depth = list(range(4, 11, 1))

# グリッドの設定
# - 引数：max_depth, パラメータのグリッド：param_depth
grid = [{'criterion': ['entropy', 'gini'], 'max_depth': param_depth}]

# グリッドサーチの作成、実行
gs = GridSearchCV(model, param_grid=grid, scoring='roc_auc', cv=5, n_jobs=-1)
gs.fit(X, Y)

# 結果の表示
print('best AUC={:.3f}'.format(gs.best_score_))
print('parameter:', gs.best_params_)

best AUC=0.673
parameter: {'criterion': 'gini', 'max_depth': 6}


In [6]:
df = pd.DataFrame(gs.cv_results_).sort_values('mean_test_score', ascending=False)
df[['mean_test_score', 'params']].head(5)

,mean_test_score,params
9,0.672550,"{'criterion': 'gini', 'max_depth': 6}"
2,0.672054,"{'criterion': 'entropy', 'max_depth': 6}"
3,0.671769,"{'criterion': 'entropy', 'max_depth': 7}"
10,0.671249,"{'criterion': 'gini', 'max_depth': 7}"
1,0.670079,"{'criterion': 'entropy', 'max_depth': 5}"
